# 1.1 - Data Assessing and Data Cleaning

___

## Project Workflow


1.0) Geological Setting and Mineral Disponibility

**1.1) Data Assessing and Data Cleaning**

1.2) Exploratory Data Analysis & Geostats

2.1) Spatial Analysis

2.2) Hydrograph Basins Delimitations

2.3) Correlation between basins and samples

3.0) Conclusion
___

## Table Of Contents

[a) Importing Libraries](#il)

[b) Importing File](#if)

[c) Data Assessing](#da)

[d) Data Cleaning](#dc)

<a name="il"></a>
## Importing Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from shapely import geometry
import matplotlib.pyplot as plt

%matplotlib inline

<a name="if"></a>
## Importing File

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
df = pd.read_csv('/content/NURE_15.csv', sep =',')
NURE_15 = df.copy()
NURE_15.head()

,rec_no,prime_id,samptyp,latitude,longitude,ornlid,site,state,quad,mapcode,...,zn_ppm,zr_ppm,po4_ppm,so4_ppm,methods,tapefile,reformat,coordprb,comments,comment2
0,5239129,8986,15,29.703,-102.759,8986,1105,TX,EMORY PEAK,NH1309,...,333,56,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,"1-38 MARAVILLAS CANYON, QUAD., 15 MIN. SAMPLE ...",
1,5239061,8874,15,29.465,-102.834,8874,1001,TX,EMORY PEAK,NH1309,...,59,64,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,SAME COORDINATES AS ORNLID(029338):,1-78 GARY HOMERSTAD % AREA MANAGER BLACK GAP W...,
2,5239062,8875,15,29.473,-102.816,8875,1002,TX,EMORY PEAK,NH1309,...,64,92,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,1-78 GARY HOMERSTAD % AREA MANAGER BLACK GAP W...,
3,5239065,8880,15,29.432,-102.863,8880,1007,TX,EMORY PEAK,NH1309,...,54,76,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,"1-38 STILLWELL CROSSING QUAD., 7.5 MIN., SAMPL...",
4,5239066,8884,15,29.871,-102.930,8884,1010,TX,EMORY PEAK,NH1309,...,19,27,0,0,"OR1, OR2, OR6, OR8",XG0386.03,HSSR SAMPLE USED IN TERRELL STUDY AREA: SAME R...,,"1-38 DOVE MTN. QUAD., 15 MIN., SAMPLE TAKEN OV...",


<a name="da"></a>
## Data Assessing

In [5]:
NURE_15.sample(10)

,rec_no,prime_id,samptyp,latitude,longitude,ornlid,site,state,quad,mapcode,...,zn_ppm,zr_ppm,po4_ppm,so4_ppm,methods,tapefile,reformat,coordprb,comments,comment2
366,5239977,29408,15,29.446,-102.868,29408,H375,TX,EMORY PEAK,NH1309,...,70,69,0,0,"OR1, OR2, OR7, OR8",XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,"1-38 STILLWELL CROSSING 7.5 MIN. TOPO., COLLEC...",
553,5330725,29210,92,29.557,-102.836,29210,H185,TX,EMORY PEAK,NH1309,...,0,0,0,0,NONE,XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,1-38 MARAVILLAS CANYON SE QUAD; READING TAKEN ...,", TX 76901:"
291,5239900,29260,15,29.508,-102.889,29260,H233,TX,EMORY PEAK,NH1309,...,71,61,0,0,"OR1, OR2, OR7, OR8",XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,"1-38 MARAVILLAS CANYON SW 7 1/2' QUAD., SAMPLE...","HADBOURNE % SAN ANGELO, TX. 76901:"
525,5330799,29498,92,29.648,-102.953,29498,H460,TX,EMORY PEAK,NH1309,...,0,0,0,0,NONE,XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,1-38 MARAVILLAS CANYON N.W. 7.5' QUAD SCINTILL...,
370,5239981,29415,15,29.487,-102.976,29415,H381,TX,EMORY PEAK,NH1309,...,59,55,0,0,"OR1, OR2, OR7, OR8",XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,1-38 SUE PEAKS 7.5 MIN QUAD COLLECTED IN THE C...,
419,5240030,29475,15,29.668,-102.881,29475,H440,TX,EMORY PEAK,NH1309,...,70,74,0,0,"OR1, OR2, OR7, OR8",XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,1-38 MARAVILLAS CANYON N.W. 7.5' QUAD COLLECTE...,LO TEXAS 76901:
573,5330765,29341,92,29.480,-102.854,29341,H311,TX,EMORY PEAK,NH1309,...,0,0,0,0,NONE,XG0386.24,STILLWELL MOUNTAINS STUDY AREA: SAMPLE NOT FOU...,,"1-38 STILLWELL CROSSING 7.5 MIN. TOPO., MEAS. ...",
629,5330831,29566,92,29.589,-102.935,29566,H529,TX,EMORY PEAK,NH1309,...,0,0,0,0,NONE,XG0386.24,STILLWELL MOUNTAINS STUDY AREA: SAMPLE NOT FOU...,,"1-38 MARAVILLAS CANYON S.W. 7.5 MIN. TOPO., ME...",
491,5240102,29635,15,29.783,-102.977,29635,H595,TX,EMORY PEAK,NH1309,...,58,86,0,0,"OR1, OR2, OR7, OR8",XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,"1-38 DOVE MOUNTAIN 15' QUAD., SAMPLE COLLECTED...",
74,5239170,10091,15,29.675,-103.024,10091,0823,TX,EMORY PEAK,NH1309,...,63,120,0,0,"OR1, OR2, OR6, OR8",XG0386.03,HSSR SAMPLE USED IN SOLITARIO STUDY AREA: SAME...,,1-78 GARY HOMERSTAD % AREA MANAGER BLACK GAP W...,


In [6]:
NURE_15.shape

(680, 137)

In [7]:
NURE_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Columns: 137 entries, rec_no to comment2
dtypes: float64(17), int64(65), object(55)
memory usage: 727.9+ KB


In [8]:
#looking for duplicated
NURE_15.duplicated().sum()

0

<a name="dc"></a>
## Data Cleaning

`1.` Tidiness Issues

- Change columns dtypes

    - Make a pattern for numbers below a certain value (e.g. <10 ppm)
    
- Filter the dataframe to only elements with high mobility

- Rename `Long__X_` and `Lat__Y_` columns


In [9]:
NURE_15.rename(columns = {'longitude':'X', 'latitude':'Y'},inplace = True)
NURE_15.head(1)

,rec_no,prime_id,samptyp,Y,X,ornlid,site,state,quad,mapcode,...,zn_ppm,zr_ppm,po4_ppm,so4_ppm,methods,tapefile,reformat,coordprb,comments,comment2
0,5239129,8986,15,29.703,-102.759,8986,1105,TX,EMORY PEAK,NH1309,...,333,56,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,"1-38 MARAVILLAS CANYON, QUAD., 15 MIN. SAMPLE ...",


___

`2.` Quality Issues

- Check for the elements that are statistically significant (>= 50% in register)

- Remove `<` or `>` sign from numbers.

In [18]:
elements = [[col for col in NURE_15.columns if col.endswith(('ppm', 'pct'))]]

In [20]:
elements.head()

AttributeError: 'list' object has no attribute 'head'

In [16]:
#Replacing - sample values with the absolute value of 1/2 to indicate analysis was preformed and sampled to 1/2 the detection limit


In [ ]:
#creating an empty list
remove = []

#iterating all elements
for e in elements:
    try:
        #counting the number of occurrences that has < or >
        minus = NURE_15[e].str.count('<').sum()
        plus = NURE_15[e].str.count('>').sum()
        #checking the percentage
        validity = 1 - (((minus + plus))/NURE_15.shape[0])

        #if the value is less than 0.5, we must remove it
        if validity < 0.5:
            del_element = e
            print(f'Remove {e}, because its value in under the proper rate {validity:0.2f}')
            #creating a list with the elements we must remove.
            remove.append(del_element)
    except:
        pass

Remove Au (ppm), because its value in under the proper rate 0.01
Remove B (ppm), because its value in under the proper rate 0.02
Remove Ge (ppm), because its value in under the proper rate 0.31
Remove Na (%), because its value in under the proper rate 0.06
Remove Re (ppm), because its value in under the proper rate 0.00
Remove S (%), because its value in under the proper rate 0.30
Remove Se (ppm), because its value in under the proper rate 0.08
Remove Ta (ppm), because its value in under the proper rate 0.10
Remove Te (ppm), because its value in under the proper rate 0.39


In [ ]:
#filtering the dataframe
NURE_15 = NURE_15.drop(columns = remove)
NURE_15.head()

,Estação,N__Lab_,X,Y,Folha,Ag (ppm),Al (%),As (ppm),Ba (ppm),Be (ppm),...,Sn (ppm),Sr (ppm),Th (ppm),Ti (%),U (ppm),V (ppm),W (ppm),Y (ppm),Zn (ppm),Zr (ppm)
0,AC-0002,CDE225,248757,7972050,Coromandel,0.03,2.63,4,76,1,...,1.5,3.3,7.4,0.03,0.70,73,0.3,13.15,27,6.3
1,AC-0003,CDE226,244460,7973135,Coromandel,0.02,1.93,2,84,1.7,...,1.6,3.1,8.2,0.06,0.94,58,0.3,23.90,58,6.9
2,AC-0004,CDE227,244044,7970217,Coromandel,0.04,1.47,3,70,0.7,...,0.9,3.9,5.4,0.04,0.65,55,0.2,10.40,34,1.8
3,AC-0005,CDE228,242895,7970593,Coromandel,0.05,1.72,23,80,1.1,...,1.1,3.1,6.9,0.04,1.01,66,0.7,9.34,27,4.1
4,AC-0006,CDE229,242999,7971416,Coromandel,0.04,0.97,7,56,0.5,...,0.7,2.5,5.7,0.05,0.82,41,0.4,7.45,28,1.5


In [ ]:
#next, let's take all < or > symbols, and convert all columns to float.

#creating another list of columns with all content that matters
all_cols = NURE_15.columns[5:]

#importing some extension apis for pandas to check the type of the column
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype


for col in all_cols:
    #if the column is numeric type, it'll pass
    if (is_numeric_dtype(NURE_15[col])):
        pass
    #if the column is string type, it'll remove symbols and the convert to float
    elif (is_string_dtype(NURE_15[col])):
        try:
            NURE_15[col] = NURE_15[col].str.replace('<','').astype('float')
        except:
            NURE_15[col] = NURE_15[col].str.replace('>','').astype('float')

The solution above was found in [this Stackoverflow post](https://stackoverflow.com/a/45568211/14649000), and was a very exquisite solution.

In [ ]:
#checking if all corrections were made.
NURE_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 47 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Estação     709 non-null    object 
 1   N__Lab_     709 non-null    object 
 2   X           709 non-null    int64  
 3   Y           709 non-null    int64  
 4   Folha       709 non-null    object 
 5   Ag (ppm)    709 non-null    float64
 6   Al (%)      709 non-null    float64
 7   As (ppm)    709 non-null    float64
 8   Ba (ppm)    709 non-null    float64
 9   Be (ppm)    709 non-null    float64
 10  Bi (ppm)    709 non-null    float64
 11  Ca (%)      709 non-null    float64
 12  Cd (ppm)    709 non-null    float64
 13  Ce (ppm)    709 non-null    float64
 14  Co (ppm)    709 non-null    float64
 15  Cr (ppm)    709 non-null    int64  
 16  Cs (ppm)    709 non-null    float64
 17  Cu (ppm)    709 non-null    float64
 18  Fe (%)      709 non-null    float64
 19  Ga (ppm)    709 non-null    f

In [ ]:
clean = NURE_15.copy()
clean.to_csv('OUTPUTS/CSV/NURE_15_data_cleaned.csv', sep = ';', index = False)

OSError: Cannot save file into a non-existent directory: 'OUTPUTS/CSV'

#### Removed Elements

A few elements were removed due to the fact that they'are not representative statisticaly.

|Element|
|-------|
|Ag|
|Au|
|B|
|Ge|
|Na|
|Re|
|S|
|Ta|
|Te|
|Ti|
|W|